# SageMaker で BYOA を行うための Docker イメージを作成する

#### ノートブックに含まれる内容

- SageMaker で BYOA(Bring Your Own Algorithm) を行うための，Docker コンテナイメージの基本的な作成のしかた

#### ノートブックで使われている手法の詳細

- Docker

## 前準備

このノートブックでは Amazon Elastic Container Registory (ECR) を使用するため，あらかじめ SageMaker を実行している IAM Role に対して，以下の IAM Policy をアタッチしてください

```
AmazonEC2ContainerRegistryFullAccess
```

## 概要

先ほどの例では，単純にノートブック上でDecision Tree アルゴリズムについて，サンプルデータをもとにモデルを学習しました．そして，学習したモデルを使って推論も行いました．ですが，ここでデータサイズが大きすぎてモデルの学習に時間がかかる場合や，できたモデルを API として公開し，大量のリクエストをさばくことができるようにしたい場合には，どうしたら良いでしょうか？

SageMaker を使うことで，このような問題を解決することができます．SageMaker は Docker コンテナを活用することにより，モデル学習や API による推論をスケーラブルな形で実行します．そのため，SageMaker を実際に使用する前に，学習および推論を行うための Docker イメージをまず最初に作成します．それから，作成した Docker イメージを使って，実際に SageMaker API 経由で学習，および推論を実行します．

このノートブックでは，scikit-learn での学習・推論を行うための Docker イメージについて説明します．以下，まず Part 1 で Docker イメージのアーキテクチャについて説明，Part 2 で実際に Docker イメージの作成を行います．

## SageMaker 用の Docker イメージの構成

### SageMaker での Docker の利用の仕方
SageMaker の Docker イメージは，学習のときと推論のときで，同じイメージを用いることができます．SageMaker の中では，学習と推論のそれぞれでコンテナを立ち上げる際に，以下のコマンドが実行されます

* 学習: `docker run $IMAGE_ID train`

* 推論: `docker run $IMAGE_ID serve`

このため，Docker イメージは `train` および `serve` というコマンドを持つ必要があります．この例では，Docker イメージ作成時に使用するスクリプト群をまとめて `container` ディレクトリに以下のように配置しました．`container/decision_trees` 内に `train` と `serve` のスクリプトが配置されているのが確認できるかと思います．これらのスクリプトは Python で書かれていますが，実際にはどの言語で書いても問題はありません．`serve` スクリプトではあくまで推論用の HTTP サーバを起動しているだけで，実際のエンドポイントの処理は `predictor.py に記述されています 

    .
    └── container
        ├── Dockerfile
        ├── build_and_push.sh
        └── decision_trees
            ├── nginx.conf
            ├── predictor.py
            ├── serve
            ├── train
            └── wsgi.py


* __`Dockerfile`__ には，Docker イメージをどのようにビルドするかが記述されています
* __`build_and_push.sh`__ は Dockerfile を使ってコンテナイメージをビルドし，ECR にプッシュするためのスクリプトです
* __`decision_trees`__ コンテナ内に含まれるファイルを配置したディレクトリです


In [ ]:
# ディレクトリの中身の確認
!ls -lR container

In [ ]:
# train スクリプトの中身の確認
!cat container/decision_trees/train

### 学習時のコンテナの実行

SageMaker が学習ジョブを走らせる際，`train` スクリプトが通常の Python プログラムのように実行されます．その際に SageMaker の仕様として，コンテナ内の `/opt/ml` ディレクトリ内に，さまざまなファイルを配置して使用する形をとります．

    /opt/ml
    ├── input
    │   ├── config
    │   │   ├── hyperparameters.json
    │   │   └── resourceConfig.json
    │   └── data
    │       └── <channel_name>
    │           └── <input data>
    ├── model
    │   └── <model files>
    └── output
        └── failure

#### インプット

* `/opt/ml/input/config` には，どのように学習処理を実行するかの情報が置かれます．`hyperparameters.json` はハイパーパラメタの名前とその値を JSON フォーマットで格納したファイルです．値は常に `string` 型として読みだされるため，その後適切な型に変換する必要があります．`resourceConfig.json` は分散学習を行う際のネットワークレイアウトを記述した JSON フォーマットのファイルです．scikit-learn では分散学習をサポートしていないため，ここ例では使用しません．
* `/opt/ml/input/data/<channel_name>/` はデータ入力方式が FILE モードのときに使われるディレクトリです．チャンネルはジョブ実行時に叩く `CreateTrainingJob` に引き渡すパラメタとして指定することができます．入力データはチャネルごとに，こちらもパラメタで指定された S3 ディレクトリからロードされたものが配置されます．
* `/opt/ml/input/data/<channel_name>_<epoch_number>` はデータ入力方式が PIPE モードのときに使われるディレクトリです．エポックは 0 から始まり順に増えていきます．ディレクトリ名はチャンネルとエポックで指定されます

#### アウトプット

* `/opt/ml/model/` は，アルゴリズムにより生成された結果のモデルが保存されるディレクトリです．モデルのフォーマットは自由に指定することができます．単一ファイルでもよいですし，階層構造を持ったディレクトリの形でも構いません．SageMaker はこのディレクトリ内のすべてのデータを圧縮済みの tar アーカイブにまとめます．このアーカイブファイルは，`DescribeTrainingJob` API のレスポンスに含まれる S3 ロケーションに置かれます
* `/opt/ml/output` にはジョブが失敗した際に，その原因が記述された `failure` ファイルが配置されます．このファイルの中身は，`DescribeTrainingJob` API のレスポンスに含まれる `FailureReason` の内容と同じです．ジョブが成功した際には，ここには何も書き出されません

### 推論時のコンテナの実行

推論時には，コンテナが API サーバとしてホストされた形で実行されます．そのため，HTTP 経由で推論のリクエストを受け付けることができます．SageMaker で API サーバをホストする際には，以下の 2 つのエンドポイントが必要です

* `/ping` はインフラからの `GET` リクエストを受けるためのエンドポイントです．リクエストを受けたら，レスポンスコード 200 を返します
 
* `/invocations` はクライアントからの `POST` 推論リクエストを受けるためのエンドポイントです．リクエストとレスポンスのフォーマットは自由に指定することができます．クライアントで `ContentType` と `Accept` ヘッダをつけた場合には，そのままエンドポイント側に引き渡されます

推論用のコンテナでは，SageMaker はモデルファイルを学習時と同じディレクトリに配置して使用します

    /opt/ml
    └── model
        └── <model files>

In [ ]:
# serve スクリプトの中身の確認
!cat container/decision_trees/predictor.py

この例では以下の図のような，私たちの推奨する構成である Python ベースの頑健かつスケーラブルなスタックを用います．

![Request serving stack](stack.png)

上記を実現するために，`container/decision_trees` ディレクトリ内には，`nginx.conf` や `wsgi.py` が配置されています．もちろん，これ以外のツールを追加したり，または全く別の構成を取ることも可能です．

    container/decision_trees
    ├── nginx.conf
    ├── predictor.py
    ├── serve
    ├── train
    └── wsgi.py

上記ファイルの中身は以下のとおりです．

* __`nginx.conf`__ は nginx の設定ファイルです
* __`predictor.py`__ は Flask による Web サーバが記述されたプログラムファイルです
* __`serve`__ は推論用コンテナとして立ち上げたときに実行されるプログラムです．中では，単に gunicorn サーバを起動して，`prediction.py` で実装された複数の Flask アプリケーションを実行します
* __`train`__ は学習用コンテナとして立ち上げたときに実行されるプログラムです．学習アルゴリズムに応じて自由に記述を変えることができます
* __`wsgi.py`__ は Flask アプリケーションを叩くための小さなラッパーです．

基本的には，上記の `train` （学習アルゴリズムの記述）と `predictor.py` （推論アルゴリズムの記述）を変更することで，実施したい機械学習の学習および推論処理を実現できます．それ以外の 3 ファイルは基本的には変更する必要はありません


## Docker イメージの作成

### Dockerfile

ここまで説明してきた仕組みを実現するために，Dockerfile でコンテナイメージの構成を定義します．

In [ ]:
!cat container/Dockerfile

### コンテナイメージをビルドして登録

以下のシェルで，`docker build` コマンドを使ってコンテナイメージをビルドし，ECR (Elastic Container Registry) にプッシュします．このスクリプトは，`container/build-and-push.sh` にシェルスクリプトとしてまとまっており，`build-and-push.sh decision_trees_sample` の形で実行することで，`decision_trees_sample` イメージを ECR にプッシュすることができます．

ECR リポジトリは，SageMaker のノートブックインスタンスがあるのと同一リージョンのものが使われます．もしリポジトリがない場合には，自動的に作られます．

以下のスクリプトを実行する前に，**<span style="color: red;">5 行目の `account_number=XX` の `XX` を指定された適切な数字に変更</span>**してください

In [ ]:
%%sh

# アルゴリズムの名前
# アカウントナンバーを修正
account_number=XX
algorithm_name=decision-trees-sample-$account_number

cd container

chmod +x decision_trees/train
chmod +x decision_trees/serve

account=$(aws sts get-caller-identity --query Account --output text)

# 現在の設定を確認して，リージョンをセット (もし定義されていない場合には，us-west-2 に設定)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# もしリポジトリが ECR に存在しない場合には作成

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# ECR からログインコマンドを取得し，直接実行
$(aws ecr get-login --region ${region} --no-include-email)

# Docker イメージを指定した名前でローカルで作成し，ECR にプッシュ
# Sagemaker ノートブックインスタンスの場合，Docker デーモンを再起動する必要がある（既知の問題で対応予定）

if [ -d "/home/ec2-user/SageMaker" ]; then
  sudo service docker restart
fi

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

### ECR のリポジトリを確認

以下の URL を開いて，作成した `decision-trees-sample-XX` のリポジトリが存在することを確認

https://console.aws.amazon.com/ecs/home?region=us-east-1#/repositories